In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
import statistics
import kmax as kx
import utility as util
from hierarchy import *
from scipy.optimize import fsolve

# Simulations

## Erdos-Renyi Graphs $(G_{n,p})$

### Approximation of $\mathbb{E}X/\mathbb{E}N$

In [ ]:
# TODO
def iterate_GWBP(lam, num_samples=100):
    Xs = 0
    Ns = 0
    for _ in range(num_samples):
        # Erdos-Reyni
        n = 100000
        sample_tree = nx.fast_gnp_random_graph(n, lam/n)

        for u,v in sample_tree.edges():
            sample_tree[u][v]['weight'] = 1

        # Calculate EX/EN for current sample
        Xs += expectation_X_sample(sample_tree)
        Ns += nx.number_of_nodes(sample_tree)
    return Xs / Ns

### Approximation of $\mathbb{E}(X/N)$

## GW Branching Process

For a Poisson branching process with $\lambda < 1$, we expect that
$$
\frac{X}{n}\underset{n\to\infty}{\longrightarrow}\frac{\mathbb{E}X}{\mathbb{E}N}
$$
where $X$ is the last-passage time of a GWBP tree with parameter $\lambda$

In theory,
$$
\mathbb{E}X = \frac{2-(\lambda p+2)e^{-\lambda p}}{1-\lambda}~~~~~\text{and}~~~~~\mathbb{E}N=\frac{1}{1-\lambda}
$$
where $p:=p(\lambda)$ is a solution to the equation $$pe^{\lambda p} = 1+\lambda p.$$

Thus
$$
\frac{\mathbb{E}X}{\mathbb{E}N} = 2-(\lambda p+2)e^{-\lambda p}.
$$
For example,
$$
\lambda = 0.7 \implies p\approx 0.874161.
$$

In [23]:
# CONSTANTS
LAM = 0.7

In [24]:
def func(p):
    return [p[0] * np.exp(LAM*p[0]) - (1 + LAM * p[0])]

In [25]:
p = fsolve(func,0)[0]
EX_EN = 2-(2+lam*p)*math.exp(-lam*p)
EX = EX_EN / (1-lam)

In [26]:
print(f'p = {p}')
print(f'EX/EN = {EX_EN}')

p = 0.8741606425132245
EX/EN = 0.583526626641014


In [30]:
def expectation_X_sample(sample_tree): 
    X = 0
    for nodes in nx.connected_components(sample_tree):
        subgraph = sample_tree.subgraph(nodes)
        path, _, Xi = util.path_cover(subgraph, list(nodes)[0])
        X += Xi / nx.number_of_nodes(subgraph)
    return X

In [31]:
def iterate_GWBP(lam, num_samples=100):
    Xs = 0
    Ns = 0
    for _ in range(num_samples):
        # Poisson Branching
        sample_tree = nx.Graph(GWBP(lam, MAXLEVEL=100))
        for u,v in sample_tree.edges():
            sample_tree[u][v]['weight'] = 1

        # Calculate EX/EN for current sample
        Xs += expectation_X_sample(sample_tree)
        Ns += nx.number_of_nodes(sample_tree)
    return Xs / Ns

In [32]:
samples_per_iteration = 1
num_iterations = 1
values = [iterate_GWBP(lam, num_samples=samples_per_iteration) for _ in range(num_iterations)]

In [33]:
mean = statistics.mean(values)
mean

0.08870471217243048

In [34]:
abs(mean-EX)

np.float64(1.8563840432976157)

In [35]:
1 - mean/EX

np.float64(0.95439554523002)

### Graphing

In [14]:
plt.plot(ns, values)
plt.xlabel("n")
plt.ylabel("EX")
plt.axhline(y=statistics.mean(values), color='red')

NameError: name 'ns' is not defined